In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px



In [60]:
df = pd.read_csv('C:/Users/SLN6CT/Desktop/IA/Data/Planilha/coffee.csv') 

In [61]:
df.isnull().sum()

date            0
datetime        0
cash_type       0
card           89
money           0
coffee_name     0
dtype: int64

In [62]:
def extrair(texto):
    if isinstance(texto, str) and len(texto) >= 19:
        return texto[11:19]
    else:
        return '00:00:00'
df['Horario'] = df['datetime'].apply(extrair)
del df['datetime']


In [63]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].dt.strftime('%d/%m/%Y')
df = df.rename(columns={'date':'Data', 'cash_type':'Pagamento', 'money': 'Valor', 'coffee_name':'Bebida', 'card':'Cartao'})

In [64]:
mapping = {
    'Tea': '1',
    'Espresso': '2', 
    'Chocolate with coffee': '3', 
    'Chocolate with milk': '4',
    'Coffee with Irish whiskey': '5', 
    'Irish whiskey with milk': '6',
    'Double Irish whiskey': '7', 
    'Double espresso': '8', 
    'Americano with milk': '9',
    'Cappuccino': '10', 
    'Double espresso with milk': '11',
    'Caramel with Irish whiskey': '12', 
    'Latte': '13', 
    'Americano': '14',
    'Super chocolate': '15', 
    'Double vanilla': '16', 
    'Mochaccino': '17',
    'Caramel with chocolate': '18', 
    'Irish whiskey': '19', 
    'Caramel coffee': '20',
    'Vanilla with Irish whiskey': '21', 
    'Chocolate': '22', 
    'Caramel with milk': '23',
    'Coffee with chocolate': '24', 
    'Irish with chocolate': '25', 
    'Vanilla coffee': '26',
    'Double ristretto': '27', 
    'Hot milkshake': '28', 
    'Double chocolate': '29', 
    'Caramel': '30',
    'Hot Chocolate': '31', 
    'Americano with Milk': '32', 
    'Cocoa': '33', 
    'Cortado': '34'
}
df['NumeroBebida'] = df['Bebida'].astype(str)
df['NumeroBebida'] = df['NumeroBebida'].map(mapping)
df['NumeroBebida'].unique()

array(['13', '31', '14', '32', '33', '34', '2', '10'], dtype=object)

In [65]:
df = df[['Data', 'Horario', 'Bebida', 'NumeroBebida', 'Valor', 'Pagamento', 'Cartao']]
df

,Data,Horario,Bebida,NumeroBebida,Valor,Pagamento,Cartao
0,01/03/2024,10:15:50,Latte,13,38.70,card,ANON-0000-0000-0001
1,01/03/2024,12:19:22,Hot Chocolate,31,38.70,card,ANON-0000-0000-0002
2,01/03/2024,12:20:18,Hot Chocolate,31,38.70,card,ANON-0000-0000-0002
3,01/03/2024,13:46:33,Americano,14,28.90,card,ANON-0000-0000-0003
4,01/03/2024,13:48:14,Latte,13,38.70,card,ANON-0000-0000-0004
...,...,...,...,...,...,...,...
3631,23/03/2025,10:34:54,Cappuccino,10,35.76,card,ANON-0000-0000-1158
3632,23/03/2025,14:43:37,Cocoa,33,35.76,card,ANON-0000-0000-1315
3633,23/03/2025,14:44:16,Cocoa,33,35.76,card,ANON-0000-0000-1315
3634,23/03/2025,15:47:28,Americano,14,25.96,card,ANON-0000-0000-1316


In [66]:
value = df[(df['Cartao'].isna())]
value = value[(value['Pagamento']=="card")]
value

,Data,Horario,Bebida,NumeroBebida,Valor,Pagamento,Cartao


In [67]:
value = df[(df['Cartao'].isna())]
value = value[(value['Pagamento']=="cash")]
value

,Data,Horario,Bebida,NumeroBebida,Valor,Pagamento,Cartao
12,02/03/2024,10:30:35,Latte,13,40.0,cash,NaN
18,03/03/2024,10:10:43,Latte,13,40.0,cash,NaN
41,06/03/2024,12:30:27,Americano with Milk,32,35.0,cash,NaN
46,07/03/2024,10:08:58,Latte,13,40.0,cash,NaN
49,07/03/2024,11:25:43,Latte,13,40.0,cash,NaN
...,...,...,...,...,...,...,...
657,31/05/2024,09:23:58,Latte,13,39.0,cash,NaN
677,01/06/2024,20:54:59,Cocoa,33,39.0,cash,NaN
685,02/06/2024,22:43:10,Americano with Milk,32,34.0,cash,NaN
691,03/06/2024,21:42:51,Americano with Milk,32,34.0,cash,NaN


In [68]:
df

,Data,Horario,Bebida,NumeroBebida,Valor,Pagamento,Cartao
0,01/03/2024,10:15:50,Latte,13,38.70,card,ANON-0000-0000-0001
1,01/03/2024,12:19:22,Hot Chocolate,31,38.70,card,ANON-0000-0000-0002
2,01/03/2024,12:20:18,Hot Chocolate,31,38.70,card,ANON-0000-0000-0002
3,01/03/2024,13:46:33,Americano,14,28.90,card,ANON-0000-0000-0003
4,01/03/2024,13:48:14,Latte,13,38.70,card,ANON-0000-0000-0004
...,...,...,...,...,...,...,...
3631,23/03/2025,10:34:54,Cappuccino,10,35.76,card,ANON-0000-0000-1158
3632,23/03/2025,14:43:37,Cocoa,33,35.76,card,ANON-0000-0000-1315
3633,23/03/2025,14:44:16,Cocoa,33,35.76,card,ANON-0000-0000-1315
3634,23/03/2025,15:47:28,Americano,14,25.96,card,ANON-0000-0000-1316
